In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-rc2'

### dataset API

https://blog.csdn.net/feixiang7701/article/details/81611356

#### dataset 的属性

dataset 可以通过element_spec查看形状和数据dtype

In [3]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(0,9))

In [8]:
dataset.element_spec

TensorSpec(shape=(), dtype=tf.int32, name=None)

In [23]:
def test_dataset_conv2d(ds,out_size,last_activation,loss):
    model = tf.keras.Sequential([
        
        tf.keras.layers.Conv2D(filters = 6,kernel_size=5,activation="sigmoid"),
        tf.keras.layers.MaxPooling2D(strides = 2),
        tf.keras.layers.Conv2D(filters = 16,kernel_size=5,activation="sigmoid"),
        tf.keras.layers.MaxPooling2D(strides = 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32,activation="sigmoid"),
        tf.keras.layers.Dense(out_size,activation=last_activation)
    ])
    
    model.compile(loss=loss,metrics=["acc"])
    
    model.fit(ds,epochs=3,steps_per_epoch=10)

#### 数据读取
可以通过from_tensor_slices,from_tensors,from_generator

In [9]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [32]:
dataset_1 = tf.data.Dataset.from_tensor_slices((x_train,y_train))

In [33]:
def map_fun(x,y):
    x = tf.divide(x,255)
    x = tf.expand_dims(x,axis=-1)
    return x,y

In [34]:
dataset_1 = dataset_1.map(map_func=map_fun).batch(256)

In [35]:
test_dataset_conv2d(dataset_1,10,last_activation="softmax",loss=tf.losses.sparse_categorical_crossentropy)

Train for 10 steps
Epoch 1/3
10/10 [==============================] - 1s 143ms/step - loss: 2.3272 - acc: 0.0965
Epoch 2/3
10/10 [==============================] - 0s 45ms/step - loss: 2.3044 - acc: 0.0918
Epoch 3/3
10/10 [==============================] - 0s 43ms/step - loss: 2.3053 - acc: 0.0930
